# Install Required Libraries (Run Once)

In [8]:
!pip install psycopg2 pandas python-dotenv


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


# Import Libraries & Load Environment Variables

In [9]:
import psycopg2
import pandas as pd
import os
from dotenv import load_dotenv

# Load environment variables from .env
load_dotenv()

True

# Secure Database Connection

In [ ]:
load_dotenv()

def get_db_connection():
    """Establish a secure PostgreSQL database connection using .env variables."""
    return psycopg2.connect(
        dbname=os.getenv("DB_NAME"),
        user=os.getenv("DB_USER"),
        password=os.getenv("DB_PASSWORD"),
        host=os.getenv("DB_HOST"),
        port=os.getenv("DB_PORT")
    )

print("Database Connected.")

# AUTHENTICATION & USER MANAGEMENT

## Create a New User

In [47]:
def create_user(role, first_name, last_name, username, email, password_hash):
    """Create a new user and return user_id. Handles duplicate usernames/emails."""
    conn = get_db_connection()
    cur = conn.cursor()
    
    try:
        cur.execute('''
            INSERT INTO "user" (role, first_name, last_name, username, email, password_hash)
            VALUES (%s, %s, %s, %s, %s, %s)
            RETURNING user_id;
        ''', (role, first_name, last_name, username, email, password_hash))
        
        user_id = cur.fetchone()[0]  # Get newly created user ID
        conn.commit()
        return user_id  # Success
    
    except psycopg2.IntegrityError as e:  # Catch duplicate entry error
        conn.rollback()  # Undo any changes
        if "unique constraint" in str(e).lower():
            print("Error: Username or email already exists.")
        else:
            print("Database Error:", e)
        return None  # Indicate failure

    finally:
        cur.close()
        conn.close()  # Ensure connection is closed

In [51]:
test_user_id = create_user(
    role="student",
    first_name="Luca",
    last_name="Antonacci",
    username="loser",
    email="loser@loser.com",
    password_hash="hashedpassword123"
)

if test_user_id:
    print(f"User created! User ID: {test_user_id}")
else:
    print("Failed to create user.")

User created! User ID: 12


## Get User Information

In [23]:
def get_user_id(username):
    """Get user ID by username"""
    conn = get_db_connection()
    cur = conn.cursor()
    cur.execute('SELECT user_id FROM "user" WHERE username = %s;', (username,))
    user_id = cur.fetchone()
    cur.close()
    conn.close()
    return user_id[0] if user_id else None

def get_user_role(user_id):
    """Retrieve user role"""
    conn = get_db_connection()
    cur = conn.cursor()
    cur.execute('SELECT role FROM "user" WHERE user_id = %s;', (user_id,))
    role = cur.fetchone()
    cur.close()
    conn.close()
    return role[0] if role else None

def get_first_name(user_id):
    """Retrieve first name"""
    conn = get_db_connection()
    cur = conn.cursor()
    cur.execute('SELECT first_name FROM "user" WHERE user_id = %s;', (user_id,))
    name = cur.fetchone()
    cur.close()
    conn.close()
    return name[0] if name else None

def get_last_name(user_id):
    """Retrieve last name"""
    conn = get_db_connection()
    cur = conn.cursor()
    cur.execute('SELECT last_name FROM "user" WHERE user_id = %s;', (user_id,))
    name = cur.fetchone()
    cur.close()
    conn.close()
    return name[0] if name else None

def get_username(user_id):
    """Retrieve username"""
    conn = get_db_connection()
    cur = conn.cursor()
    cur.execute('SELECT username FROM "user" WHERE user_id = %s;', (user_id,))
    username = cur.fetchone()
    cur.close()
    conn.close()
    return username[0] if username else None

def get_email(user_id):
    """Retrieve email"""
    conn = get_db_connection()
    cur = conn.cursor()
    cur.execute('SELECT email FROM "user" WHERE user_id = %s;', (user_id,))
    email = cur.fetchone()
    cur.close()
    conn.close()
    return email[0] if email else None

def get_week_activity(user_id):
    """Retrieve user's weekly activity (bit string tracking 7 days)."""
    conn = get_db_connection()
    cur = conn.cursor()
    cur.execute('SELECT week_activity FROM "user" WHERE user_id = %s;', (user_id,))
    activity = cur.fetchone()
    cur.close()
    conn.close()
    return activity[0] if activity else None

In [24]:
print(f"User ID: {get_user_id('testuser123')}")
print(f"Role: {get_user_role(test_user_id)}")
print(f"First Name: {get_first_name(test_user_id)}")
print(f"Last Name: {get_last_name(test_user_id)}")
print(f"Username: {get_username(test_user_id)}")
print(f"Email: {get_email(test_user_id)}")
print(f"Weekly Acttivity: {get_week_activity(test_user_id)}")

User ID: None
Role: student
First Name: Test
Last Name: User
Username: updateduser123
Email: testuser@example.com
Weekly Acttivity: 0000000


## Update User Information

In [49]:
def update_username(user_id, new_username):
    """Update user's username"""
    conn = get_db_connection()
    cur = conn.cursor()
    cur.execute('UPDATE "user" SET username = %s WHERE user_id = %s;', (new_username, user_id))
    conn.commit()
    cur.close()
    conn.close()

def update_email(user_id, new_email):
    """Update a user's email address"""
    conn = get_db_connection()
    cur = conn.cursor()
    
    cur.execute('''
        UPDATE "user"
        SET email = %s
        WHERE user_id = %s;
    ''', (new_email, user_id))
    
    conn.commit()
    cur.close()
    conn.close()

def update_week_activity(user_id, day_index):
    """Set a specific day as active in the week activity BIT(7) column"""
    conn = get_db_connection()
    cur = conn.cursor()

    # Create bitmask for the day (e.g., 0010000 for Wednesday)
    bitmask = 1 << (6 - day_index)  # Bitwise shift to set the correct bit

    # Update using bitwise OR to preserve previous activity
    cur.execute('''
        UPDATE "user"
        SET week_activity = week_activity | %s::bit(7)
        WHERE user_id = %s;
    ''', (format(bitmask, '07b'), user_id))

    conn.commit()
    cur.close()
    conn.close()

def reset_week_activity():
    """Reset all users' week activity back to 0000000 at the start of a new week."""
    conn = get_db_connection()
    cur = conn.cursor()
    
    # Set week_activity to all zeros for every user
    cur.execute('''
        UPDATE "user"
        SET week_activity = B'0000000';
    ''')
    
    conn.commit()
    cur.close()
    conn.close()

def delete_user(user_id):
    """Delete a user by user_id."""
    conn = get_db_connection()
    cur = conn.cursor()

    # Check if the user exists before deleting
    cur.execute('SELECT user_id FROM "user" WHERE user_id = %s;', (user_id,))
    existing_user = cur.fetchone()

    if not existing_user:
        print("Error: User not found.")
        return False  # Indicate failure

    # Delete the user
    cur.execute('DELETE FROM "user" WHERE user_id = %s;', (user_id,))
    conn.commit()

    cur.close()
    conn.close()

In [50]:
update_username(test_user_id, "updateduser123")
print(f"Updated Username: {get_username(test_user_id)}")

Updated Username: None


In [42]:
update_email(test_user_id, "newemail@example.com")
print(f"Updated Email: {get_email(test_user_id)}")

Updated Email: newemail@example.com


In [38]:
print(f"Initial Weekly Activity: {get_week_activity(test_user_id)}")
update_week_activity(test_user_id, 0)
print(f"Updated Weekly Activity (M Active): {get_week_activity(test_user_id)}")
update_week_activity(test_user_id, 2)
print(f"Updated Weekly Activity (M + W Active): {get_week_activity(test_user_id)}")

# import schedule
# import time

# # Schedule weekly reset every Monday at 12:01 AM
# schedule.every().monday.at("00:01").do(reset_weekly_activity)

# while True:
#     schedule.run_pending()
#     time.sleep(60)  # Check every minute

reset_week_activity()
print(f"After Reset: {get_week_activity(test_user_id)}")

Initial Weekly Activity: 0000000
Updated Weekly Activity (M Active): 1000000
Updated Weekly Activity (M + W Active): 1010000
After Reset: 0000000


In [55]:
delete_success = delete_user(12)
if delete_success:
    print("User was successfully deleted.")
else:
    print("Failed to delete user.")

Error: User not found.
Failed to delete user.


# CLASS MANAGEMENT

In [81]:
def create_class(class_name, owner_id):
    """Create a new class and assign the owner in user_class_roster."""
    conn = get_db_connection()
    cur = conn.cursor()

    # Step 1: Insert the class into the class table
    cur.execute('''
        INSERT INTO "class" (class_name, class_code, is_active, created_at)
        VALUES (%s, LEFT(MD5(random()::text), 8), TRUE, CURRENT_TIMESTAMP)
        RETURNING class_id;
    ''', (class_name,))
    
    class_id = cur.fetchone()[0]  # Get the new class ID

    # Step 2: Insert the owner into the user_class_roster table
    cur.execute('''
        INSERT INTO "user_class_roster" (user_id, class_id, role, enrollment_date)
        VALUES (%s, %s, 'owner', CURRENT_DATE);
    ''', (owner_id, class_id))

    conn.commit()
    cur.close()
    conn.close()
    return class_id

def get_active_classes():
    """Retrieve all active classes."""
    conn = get_db_connection()
    cur = conn.cursor()
    cur.execute('SELECT * FROM "class" WHERE is_active = TRUE;')
    classes = cur.fetchall()
    cur.close()
    conn.close()
    return classes

def get_class_by_id(class_id):
    """Retrieve a specific class by its ID."""
    conn = get_db_connection()
    cur = conn.cursor()
    cur.execute('SELECT * FROM "class" WHERE class_id = %s;', (class_id,))
    class_info = cur.fetchone()
    cur.close()
    conn.close()
    return class_info

def get_class_code(class_id):
    """Retrieve the class code for a specific class."""
    conn = get_db_connection()
    cur = conn.cursor()
    cur.execute('SELECT class_code FROM "class" WHERE class_id = %s;', (class_id,))
    class_code = cur.fetchone()[0]
    cur.close()
    conn.close()
    return class_code

def join_class_by_code(student_id, class_code):
    """Enroll a student in a class using the class code."""
    conn = get_db_connection()
    cur = conn.cursor()
    
    # Check if class code exists
    cur.execute('SELECT class_id FROM "class" WHERE class_code = %s;', (class_code,))
    result = cur.fetchone()
    
    if result:
        class_id = result[0]
        
        # Add student to class roster
        cur.execute('''
            INSERT INTO "user_class_roster" (user_id, class_id, role, enrollment_date)
            VALUES (%s, %s, 'participant', CURRENT_DATE);
        ''', (student_id, class_id))
        
        conn.commit()
        print(f"Student {student_id} joined class {class_id} successfully.")
    else:
        print("Error: Invalid class code.")
    
    cur.close()
    conn.close()

def archive_class(class_id):
    """Archive a class (set is_active to FALSE)."""
    conn = get_db_connection()
    cur = conn.cursor()
    cur.execute('UPDATE "class" SET is_active = FALSE WHERE class_id = %s;', (class_id,))
    conn.commit()
    cur.close()
    conn.close()

def delete_class(class_id):
    """Delete a class by first removing all linked users from user_class_roster."""
    conn = get_db_connection()
    cur = conn.cursor()

    # Remove all students/owners from user_class_roster
    cur.execute('DELETE FROM "user_class_roster" WHERE class_id = %s;', (class_id,))

    # Delete the class safely
    cur.execute('DELETE FROM "class" WHERE class_id = %s;', (class_id,))
    
    conn.commit()
    cur.close()
    conn.close()

In [59]:
current_user_id = get_user_id("testuser123")
new_class_id = create_class("Math 101", current_user_id)
print(f"Created Class ID: {new_class_id} for User ID: {current_user_id}")

Created Class ID: 1 for User ID: 9


In [62]:
print(f"Active Classes: {get_active_classes()}")

Active Classes: [(1, 'Math 101', '3e63e766', True, datetime.datetime(2025, 3, 12, 13, 29, 46, 665901))]


In [64]:
print(f"Class Info: {get_class_by_id(1)}")

Class Info: (1, 'Math 101', '3e63e766', True, datetime.datetime(2025, 3, 12, 13, 29, 46, 665901))


In [72]:
print(f"Class Code: {get_class_code(1)}")

Class Code: 3e63e766


In [74]:
join_class_by_code(4, get_class_code(new_class_id))

Student 4 joined class 1 successfully.


In [76]:
archive_class(new_class_id)
print(f"Archived Class: {get_class_by_id(new_class_id)}")

Archived Class: (1, 'Math 101', '3e63e766', False, datetime.datetime(2025, 3, 12, 13, 29, 46, 665901))


In [80]:
delete_class(new_class_id)
print(f"Deleted Class: {get_class_by_id(new_class_id)}")

Deleted Class: None


# COURSE MANAGEMENT

In [88]:
def create_private_course(course_name, owner_id):
    """Create a new private course."""
    conn = get_db_connection()
    cur = conn.cursor()
    cur.execute('''
        INSERT INTO "course" (course_name, owner_id, permission)
        VALUES (%s, %s, 'private')
        RETURNING course_id;
    ''', (course_name, owner_id))
    course_id = cur.fetchone()[0]
    conn.commit()
    cur.close()
    conn.close()
    return course_id

def create_public_course(course_name, owner_id):
    """Create a new public course."""
    conn = get_db_connection()
    cur = conn.cursor()
    cur.execute('''
        INSERT INTO "course" (course_name, owner_id, permission)
        VALUES (%s, %s, 'public')
        RETURNING course_id;
    ''', (course_name, owner_id))
    course_id = cur.fetchone()[0]
    conn.commit()
    cur.close()
    conn.close()
    return course_id

def delete_course(course_id):
    """Delete a course after removing dependencies."""
    conn = get_db_connection()
    cur = conn.cursor()
    
    # Remove linked records before deleting the course
    cur.execute('DELETE FROM "course_class_mapping" WHERE course_id = %s;', (course_id,))
    cur.execute('DELETE FROM "user_course_enrollment" WHERE course_id = %s;', (course_id,))
    cur.execute('DELETE FROM "module_course_mapping" WHERE course_id = %s;', (course_id,))
    
    # Delete the course
    cur.execute('DELETE FROM "course" WHERE course_id = %s;', (course_id,))
    
    conn.commit()
    cur.close()
    conn.close()

In [89]:
private_course_id = create_private_course("Advanced Python", 4)
print(f"Created Private Course ID: {private_course_id}")

Created Private Course ID: 5


In [90]:
public_course_id = create_public_course("Intro to JavaScript", 4)
print(f"Created Public Course ID: {public_course_id}")

Created Public Course ID: 6


In [91]:
delete_course(private_course_id)
print(f"Deleted Private Course: {private_course_id}")

Deleted Private Course: 5


# MODULE MANAGEMENT

In [92]:
def create_module(module_name, owner_id):
    """Create a new module."""
    conn = get_db_connection()
    cur = conn.cursor()
    cur.execute('''
        INSERT INTO "module" (module_name, status, owner_id)
        VALUES (%s, 'locked', %s)
        RETURNING module_id;
    ''', (module_name, owner_id))
    module_id = cur.fetchone()[0]
    conn.commit()
    cur.close()
    conn.close()
    return module_id

def add_module_to_course(course_id, module_id, module_order):
    """Add a module to a course with a specified order."""
    conn = get_db_connection()
    cur = conn.cursor()
    cur.execute('''
        INSERT INTO "module_course_mapping" (course_id, module_id, module_order)
        VALUES (%s, %s, %s);
    ''', (course_id, module_id, module_order))
    conn.commit()
    cur.close()
    conn.close()

def delete_module(module_id):
    """Delete a module after removing dependencies."""
    conn = get_db_connection()
    cur = conn.cursor()
    cur.execute('DELETE FROM "module_course_mapping" WHERE module_id = %s;', (module_id,))
    cur.execute('DELETE FROM "user_module_progress" WHERE module_id = %s;', (module_id,))
    cur.execute('DELETE FROM "module" WHERE module_id = %s;', (module_id,))
    conn.commit()
    cur.close()
    conn.close()

In [100]:
module_id = create_module("Object-Oriented Programming", 4)
print(f"Created Module ID: {module_id}")

Created Module ID: 4


In [101]:
add_module_to_course(public_course_id, module_id, 2)
print(f"Added Module {module_id} to Course {public_course_id} at Order 2")

Added Module 4 to Course 6 at Order 2


In [99]:
delete_module(module_id)
print(f"Deleted Module ID: {module_id}")

Deleted Module ID: 3


# BADGES

In [105]:
def create_badge(badge_name, badge_description, s3_url):
    """Create a new badge."""
    conn = get_db_connection()
    cur = conn.cursor()
    cur.execute('''
        INSERT INTO "badge" (badge_name, badge_description, s3_url)
        VALUES (%s, %s, %s)
        RETURNING badge_id;
    ''', (badge_name, badge_description, s3_url))
    badge_id = cur.fetchone()[0]
    conn.commit()
    cur.close()
    conn.close()
    return badge_id

def award_badge(user_id, badge_id):
    """Award a badge to a user."""
    conn = get_db_connection()
    cur = conn.cursor()
    cur.execute('''
        INSERT INTO "user_badge_achievement" (user_id, badge_id, earned_date)
        VALUES (%s, %s, CURRENT_TIMESTAMP);
    ''', (user_id, badge_id))
    conn.commit()
    cur.close()
    conn.close()

def get_user_badges(user_id):
    """Retrieve all badges earned by a user."""
    conn = get_db_connection()
    cur = conn.cursor()
    cur.execute('''
        SELECT b.badge_id, b.badge_name, b.badge_description, b.s3_url, uba.earned_date
        FROM "badge" b
        JOIN "user_badge_achievement" uba ON b.badge_id = uba.badge_id
        WHERE uba.user_id = %s;
    ''', (user_id,))
    badges = cur.fetchall()
    cur.close()
    conn.close()
    return badges

In [103]:
badge_id = create_badge("Python Master", "Earned by completing all Python courses", "https://s3.amazonaws.com/badges/python_master.png")
print(f"Created Badge ID: {badge_id}")

Created Badge ID: 1


In [104]:
award_badge(4, badge_id)
print(f"Awarded Badge {badge_id} to User 1")

Awarded Badge 1 to User 1


In [107]:
user_badges = get_user_badges(4)
print(f"✅ Badges for User 4: {user_badges}")

✅ Badges for User 4: [(1, 'Python Master', 'Earned by completing all Python courses', 'https://s3.amazonaws.com/badges/python_master.png', datetime.datetime(2025, 3, 12, 15, 7, 46, 495473))]


# PROJECT MANAGEMENT

In [110]:
def create_project(user_id, project_name, module_id, s3_url):
    """Create a new project."""
    conn = get_db_connection()
    cur = conn.cursor()
    cur.execute('''
        INSERT INTO "project" (user_id, project_name, module_id, s3_url)
        VALUES (%s, %s, %s, %s)
        RETURNING project_id;
    ''', (user_id, project_name, module_id, s3_url))
    project_id = cur.fetchone()[0]
    conn.commit()
    cur.close()
    conn.close()
    return project_id

def update_project(project_id, new_project_name):
    """Update project name and timestamp."""
    conn = get_db_connection()
    cur = conn.cursor()
    cur.execute('''
        UPDATE "project"
        SET project_name = %s, last_modified = CURRENT_TIMESTAMP
        WHERE project_id = %s;
    ''', (new_project_name, project_id))
    conn.commit()
    cur.close()
    conn.close()

def delete_project(project_id):
    """Delete a project."""
    conn = get_db_connection()
    cur = conn.cursor()
    cur.execute('DELETE FROM "project" WHERE project_id = %s;', (project_id,))
    conn.commit()
    cur.close()
    conn.close()

In [111]:
project_id = create_project(4, "Final Python Project", module_id, "https://s3.amazonaws.com/projects/final_python_project.zip")
print(f"Created Project ID: {project_id}")

Created Project ID: 1


In [112]:
update_project(project_id, "Updated Python Project")
print(f"Updated Project ID: {project_id}")

Updated Project ID: 1


In [113]:
delete_project(project_id)
print(f"Deleted Project ID: {project_id}")

Deleted Project ID: 1
